## Introduction

## Data Description

## Preregistration of Hypotheses

Hypothesis 1: If a country’s status is developing, then they will have greater borrower’s obligation compared to a developed country. 

Analysis: We will conduct a logistic regression where we input whether or not a country is developed or developing (a dummy variable) and output the borrower’s obligation (in USD dollars). The developed country dummy will be the reference variable, so we will test whether e, where  is the coefficient output by the regression equation for the ‘developing’ variable, is greater than 0.

Hypothesis 2: The higher a country’s GDP (in USD Dollars), the higher the intellectual property charges (in USD Dollars). 

Analysis: We will conduct a linear regression where we input the GDP of a country and output the intellectual property charges. We would test whether  (the regression coefficient of GDP in our equation) is greater than 0 (if an increase in GDP causes an increase in Intellectual Property Charges). 


## Data Analysis 

## Evaluation of Significance

## Intepretation and Conclusions

## Limitations